In [1]:
import keras
from keras.models import Sequential
from keras.layers import *
import pandas as pd
from keras.preprocessing import image
import os

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset=pd.read_excel("EvaluationSet/NaturalBlurSet.xlsx")

In [3]:
# My LeNet architecture
model = Sequential()

# conv filters of 5x5 each

# Layer 1
model.add(Convolution2D(32, 5, 5, input_shape=(192, 192, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
model.add(Convolution2D(32, 5, 5))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))    

model.add(Dropout(0.25))

# Layer 3
model.add(Convolution2D(64, 5, 5))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))    


model.add(Flatten())

# Layer 4
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))


# Layer 5
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))

# Layer 6
model.add(Dense(2))
model.add(Activation("softmax"))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(192, 192,...)`
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5))`
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5))`


In [4]:
model.load_weights('model_weights.h5')

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 188, 188, 32)      2432      
_________________________________________________________________
activation_1 (Activation)    (None, 188, 188, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 94, 94, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 90, 32)        25632     
_________________________________________________________________
activation_2 (Activation)    (None, 90, 90, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 45, 45, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 45, 45, 32)        0         
__________

In [6]:
from scipy.misc import imresize
from scipy.misc import imread


temp_pred=[]
temp_orig=[]
for index, row in dataset.iterrows():
    img_name=row["Image Name"]+'.jpg'
    img_path=os.path.join('EvaluationSet/NaturalBlurSet',img_name)
    img=imread(img_path)
    img=imresize(img,(192,192))
    img=img.astype('float32')
    temp_pred.append(model.predict_classes(img.reshape(1,192,192,3)))
    temp_orig.append(row["Blur Label"])

import numpy as np

prediction=np.stack(temp_pred)
original=np.stack(temp_orig)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()


In [7]:
prediction[prediction==0]=-1

In [8]:
from sklearn.metrics import accuracy_score

accuracy_score(prediction,original)

0.614

In [9]:
from sklearn.metrics import classification_report

print classification_report(original,prediction,target_names=['Clear',"Blurred"])

             precision    recall  f1-score   support

      Clear       0.70      0.60      0.65       589
    Blurred       0.52      0.64      0.58       411

avg / total       0.63      0.61      0.62      1000

